In [1]:
from sage.all import MixedIntegerLinearProgram

In [2]:
p = MixedIntegerLinearProgram()

In [3]:
x, y, z = p['x'], p['y'], p['z']

In [4]:
p.set_min(x, 0)
p.set_min(y, 0)
p.set_min(z, 0)

In [5]:
p.set_objective(x + y + 3 * z)

In [6]:
p.add_constraint(x + 2 * y <= 4)
p.add_constraint(5 * z - y <= 8)

In [7]:
p.solve()

8.8

In [8]:
p.get_values(x), p.get_values(y), p.get_values(z)

(4.0, 0.0, 1.6)

In [9]:
%load_ext autoreload

In [10]:
%autoreload 2

In [11]:
from juggling_dlx_milp import *

In [12]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [13]:
balls, throws = music_to_throws(music)

In [14]:
balls

{'do', 'mi', 're'}

In [15]:
throws

[[Throw(ball='do', time=0, max_height=1),
  Throw(ball='re', time=0, max_height=4),
  Throw(ball='mi', time=0, max_height=5)],
 [Throw(ball='do', time=1, max_height=1)],
 [Throw(ball='do', time=2, max_height=1)],
 [Throw(ball='do', time=3, max_height=6)],
 [Throw(ball='re', time=4, max_height=3)],
 [Throw(ball='mi', time=5, max_height=5)],
 [],
 [Throw(ball='re', time=7, max_height=4)],
 [],
 [Throw(ball='do', time=9, max_height=4)],
 [],
 [Throw(ball='re', time=11, max_height=1)],
 []]

In [16]:
ec_instance = throws_to_extended_exact_cover(balls, throws, 2, 4, 2,
                                             [(1, 3)], [(1, 3)])

In [17]:
# des erreurs vont être affichées mais c'est bon en fait
generate_full_table(ec_instance)

In [24]:
latex_table(ec_instance)

'\\begin{array}{ccccc}\\hline x_{T(do, 0, 1)}^{0, 1} & l_{T(do, 0, 1)} & i_{0, 0, 1} & i_{1, 0, 3} & m_{0, 0, (1, 3)}\\\\x_{T(do, 0, 1)}^{1, 1} & l_{T(do, 0, 1)} & i_{0, 1, 1} & i_{1, 1, 3} & m_{0, 1, (1, 3)}\\\\x_{T(re, 0, 4)}^{0, 1} & l_{T(re, 0, 4)} & i_{1, 0, 3} & m_{3, 0, (1, 3)}\\\\x_{T(re, 0, 4)}^{0, 2} & l_{T(re, 0, 4)}\\\\x_{T(re, 0, 4)}^{0, 3} & l_{T(re, 0, 4)} & m_{1, 0, (1, 3)}\\\\x_{T(re, 0, 4)}^{0, 4} & l_{T(re, 0, 4)} & i_{0, 0, 4}\\\\x_{T(re, 0, 4)}^{1, 1} & l_{T(re, 0, 4)} & i_{1, 1, 3} & m_{3, 1, (1, 3)}\\\\x_{T(re, 0, 4)}^{1, 2} & l_{T(re, 0, 4)}\\\\x_{T(re, 0, 4)}^{1, 3} & l_{T(re, 0, 4)} & m_{1, 1, (1, 3)}\\\\x_{T(re, 0, 4)}^{1, 4} & l_{T(re, 0, 4)} & i_{0, 1, 4}\\\\x_{T(mi, 0, 5)}^{0, 1} & l_{T(mi, 0, 5)} & i_{1, 0, 3} & m_{4, 0, (1, 3)}\\\\x_{T(mi, 0, 5)}^{0, 2} & l_{T(mi, 0, 5)}\\\\x_{T(mi, 0, 5)}^{0, 3} & l_{T(mi, 0, 5)} & m_{2, 0, (1, 3)}\\\\x_{T(mi, 0, 5)}^{0, 4} & l_{T(mi, 0, 5)}\\\\x_{T(mi, 0, 5)}^{1, 1} & l_{T(mi, 0, 5)} & i_{1, 1, 3} & m_{4, 1, (1, 3)}\\\

In [25]:
generate_table(ec_instance)